In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
# Now let's try to expand it to the whole run
#start = Time("2023-01-26T06:51:00", scale='utc')
#end = Time("2023-01-26T09:50:00", scale='utc')
# Second more recent run
#start = Time("2023-03-23T06:50:00", scale='utc')
#end = Time("2023-03-23T09:15:00", scale='utc')
# Accel correlation
start = Time("2023-03-25T01:26:29", scale='utc')
end = Time("2023-03-25T01:46:02", scale='utc')

az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['*'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['*'],  start, end)    

az_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['azimuth', 'taiTime'],  start, end)
el_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['elevation', 'taiTime'],  start, end)   
print(len(az_track), len(el_track))

In [ ]:
az['actualPosition'].plot()

In [ ]:
azPos = await client.select_time_series('lsst.sal.MTMount.logevent_azimuthInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
azPos = azPos[azPos['inPosition']] # Select only the True values
elPos = await client.select_time_series('lsst.sal.MTMount.logevent_elevationInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
elPos = elPos[elPos['inPosition']] # Select only the True values
print(len(azPos), len(elPos))

In [ ]:
# Find all of the time stamps

# Start with start_slew times

azs = az_track.values[:,0]
els = el_track.values[:,0]
times = az_track.values[:,1]
start_slew_times_1 = []
slew_dist_1 = []
slew_times_1 = []
for i in range(1,len(az_track)):
    az_shift = abs(azs[i] - azs[i-1])
    el_shift = abs(els[i] - els[i-1])
    if (az_shift > 0.1) or (el_shift > 0.1):
        start_slew_times_1.append(times[i])
        az_shift_mod = az_shift * np.cos(els[i]*np.pi/180.0)
        shift = np.sqrt(el_shift*el_shift + az_shift_mod*az_shift_mod)
        slew_dist_1.append(shift)
print(len(start_slew_times_1))

# Now in position timestamps

inPos_1 = []
azPosValues = azPos.values[:,1]
elPosValues = elPos.values[:,1]

# Subtract 1 second from the inPos values, because the mount has stabilized well before the inPos signal
for i in range(len(azPos)):
    if azPosValues[i] > elPosValues[i]:
        inPos_1.append(azPosValues[i])
    else:
        inPos_1.append(elPosValues[i])

print(len(inPos_1))

# Now pair them up

pairMin = 1.0
pairMax = 6.0

start_slew_times = []
slew_dist = []
slew_times = []
inPos = []

for i in range(len(start_slew_times_1)):
    for j in range(len(inPos_1)):
        deltaT = inPos_1[j] - start_slew_times_1[i] 
        if deltaT > pairMin and deltaT < pairMax:
            inPos.append(inPos_1[j])
            start_slew_times.append(start_slew_times_1[i])
            slew_times.append(deltaT)
            slew_dist.append(slew_dist_1[i])
        
print(len(inPos), len(start_slew_times), len(slew_times), len(slew_dist))

In [ ]:
# Plotting the mount plots
maxAzVels = []
maxAzAccels = []
maxAzJerks = []
maxElVels = []
maxElAccels = []
maxElJerks = []
smoothingFactor = 0.2 # In spline creation
kernel_size = 100 # In convolution
kernel = np.ones(kernel_size) / kernel_size

fig = plt.figure(figsize = (8,8))
for index in [1]:#range(len(inPos)):

    plotAz = az[(az['timestamp'] > (start_slew_times[index]-2.0)) & (az['timestamp'] < (inPos[index]))]
    plotEl = el[(el['timestamp'] > (start_slew_times[index]-2.0)) & (el['timestamp'] < (inPos[index]))]
    
    ss_time = Time(start_slew_times[index], format='unix_tai', scale='utc').isot
    ip_time = Time(inPos[index], format='unix_tai', scale='utc').isot
    
    # Now calculates the spline fit and differentiate it to get the acceleration and jerk
    azPs = plotAz['actualPosition'].values
    azVs = plotAz['actualVelocity'].values
    azXs = plotAz['timestamp'].values# - plotAz['timestamp'].values[0]  
    elPs = plotEl['actualPosition'].values
    elVs = plotEl['actualVelocity'].values
    elXs = plotEl['timestamp'].values# - plotEl['timestamp'].values[0]
    plotStart = azXs[0] + 1.0
    plotEnd = azXs[-1] - 1.0
    
    
    plotAzXs = np.linspace(azXs[0], azXs[-1], 2000)
    plotElXs = np.linspace(elXs[0], elXs[-1], 2000)
    azPSpline = UnivariateSpline(azXs, azPs, s=0)
    azVelSpline1 = UnivariateSpline(azXs, azVs, s=0) 
    #azVelSpline1 =azPSpline.derivative(n=1)
    # Now smooth the derivative before differentiating again
    smoothedAzVel = np.convolve(azVelSpline1(plotAzXs), kernel, mode='same')
    azVelSpline = UnivariateSpline(plotAzXs, smoothedAzVel, s=smoothingFactor)
    azAccSpline1 = azVelSpline.derivative(n=1)
    smoothedAzAcc = np.convolve(azAccSpline1(plotAzXs), kernel, mode='same')
    # Now smooth the derivative before differentiating again
    azAccSpline = UnivariateSpline(plotAzXs, smoothedAzAcc, s=smoothingFactor)
    azJerkSpline = azAccSpline.derivative(n=1) 
    elPSpline = UnivariateSpline(elXs, elPs, s=0)
    elVelSpline1 = UnivariateSpline(elXs, elVs, s=0)
    #elVelSpline1 =elPSpline.derivative(n=1)
    # Now smooth the derivative before differentiating again
    smoothedElVel = np.convolve(elVelSpline1(plotElXs), kernel, mode='same')
    elVelSpline = UnivariateSpline(plotElXs, smoothedElVel, s=smoothingFactor)
    elAccSpline1 = elVelSpline.derivative(n=1)
    smoothedElAcc = np.convolve(elAccSpline1(plotElXs), kernel, mode='same')
    # Now smooth the derivative before differentiating again
    elAccSpline = UnivariateSpline(plotElXs, smoothedElAcc, s=smoothingFactor)
    elJerkSpline = elAccSpline.derivative(n=1) 

    
    fig.clear()
    plt.subplots_adjust(wspace=0.3, hspace=0.5)
    plt.suptitle(f"MT Mount Slews - {ip_time}", fontsize = 18)
    plt.subplot(4,2,1)
    plt.plot(plotAzXs, azPSpline(plotAzXs), lw=3, color='r', label='Spline fit')
    plt.scatter(azXs, azPs, marker='x', color='red', s=100, label='Measured points')
    plt.title(f"Azimuth")
    plt.ylabel("Degrees")
    plt.xlim(plotStart, plotEnd)
    plt.legend()
    plt.subplot(4,2,2)
    plt.plot(plotElXs, elPSpline(plotElXs), lw=3, color='g', label='Spline fit')
    plt.scatter(elXs, elPs, marker='x', color='g', s=100, label='Measured points')
    plt.title(f"Elevation")
    plt.ylabel("Degrees")
    plt.xlim(plotStart, plotEnd)
    plt.legend()
    plt.subplot(4,2,3)
    plt.plot(plotAzXs, azVelSpline(plotAzXs), lw=3, color='r', label='Spline fit')
    plt.scatter(azXs, azVs, marker='x', color='red', s=100, label='Measured points')
    plt.title(f"Azimuth Velocity")
    plt.ylabel("Degrees/sec")
    plt.xlim(plotStart, plotEnd)
    plt.legend()
    plt.subplot(4,2,4)
    plt.plot(plotElXs, elVelSpline(plotElXs), lw=3, color='g', label='Spline fit')
    plt.scatter(elXs, elVs, marker='x', color='g', s=100, label='Measured points')
    plt.title(f"Elevation Velocity")
    plt.ylabel("Degrees/sec")
    plt.xlim(plotStart, plotEnd)
    plt.legend()
    plt.subplot(4,2,5)
    plt.plot(plotAzXs, azAccSpline(plotAzXs), lw=3, color='r', label='Acceleration')
    plt.title(f"Azimuth Acceleration")
    plt.ylabel("Degrees/sec^2")
    plt.xlim(plotStart, plotEnd)
    plt.legend()
    plt.subplot(4,2,6)
    plt.plot(plotElXs, elAccSpline(plotElXs), lw=3, color='g', label='Acceleration')
    plt.title(f"Elevation Acceleration")
    plt.ylabel("Degrees/sec^2")
    plt.xlim(plotStart, plotEnd)
    plt.legend()
    plt.subplot(4,2,7)
    plt.plot(plotAzXs, azJerkSpline(plotAzXs), lw=3, color='r', label='Jerk')
    plt.title(f"Azimuth Jerk")
    plt.ylabel("Degrees/sec^3")
    plt.xlim(plotStart, plotEnd)
    plt.legend()
    plt.subplot(4,2,8)
    plt.plot(plotElXs, elJerkSpline(plotElXs), lw=3, color='g', label='Jerk')
    plt.title(f"Elevation Jerk")
    plt.ylabel("Degrees/sec^3")
    plt.xlim(plotStart, plotEnd)
    plt.legend()
    timestamp = ip_time.split('.')[0].replace('-','').replace(':','')
    plt.savefig(f"/home/craiglagegit/DATA/MTMount_26jan23/MT_Mount_Accel_Jerk_{timestamp}.pdf")
    maxAzVel = np.max(abs(azVelSpline(plotAzXs)))
    maxAzAccel = np.max(abs(azAccSpline(plotAzXs)))
    maxAzJerk = np.max(abs(azJerkSpline(plotAzXs)))
    maxElVel = np.max(abs(elVelSpline(plotAzXs)))
    maxElAccel = np.max(abs(elAccSpline(plotAzXs)))
    maxElJerk = np.max(abs(elJerkSpline(plotAzXs)))
    maxAzVels.append(maxAzVel)
    maxAzAccels.append(maxAzAccel)
    maxAzJerks.append(maxAzJerk)
    maxElVels.append(maxElVel)
    maxElAccels.append(maxElAccel)
    maxElJerks.append(maxElJerk)
    if maxAzJerk > 42.0 or maxElJerk > 21.0 or maxAzAccel > 10.5 or maxElAccel > 5.25:
        print(maxAzVel, maxAzAccel, maxAzJerk, maxElVel, maxElAccel, maxElJerk)
        plt.savefig(f"/home/craiglagegit/DATA/MTMount_accel_jerk_fails/MT_Mount_Accel_Jerk_{timestamp}.pdf")
        print(timestamp)
    


In [ ]:
fig = plt.figure(figsize=(8,8))
plt.subplots_adjust(wspace=0.3, hspace=0.5)
plt.suptitle(f"MT Mount Accels and Jerks", fontsize = 18)
plt.subplot(2,3,1)
plt.title("Az Max Velocity")
plt.hist(maxAzVels, color='orange')
plt.xlabel("Velocity(deg/s)")
plt.ylim(0,120)
plt.plot([10.5, 10.5],[0,120],color='red', ls='--', label='Max')
plt.plot([7.0, 7.0],[0,120],color='blue', ls='--', label='Design')
plt.legend()
plt.subplot(2,3,2)
plt.title("Az Max Accels")
plt.hist(maxAzAccels, color='orange')
plt.xlabel("Accel(deg/s^2)")
plt.ylim(0,120)
plt.plot([10.5, 10.5],[0,120],color='red', ls='--', label='Max')
plt.plot([7.0, 7.0],[0,120],color='blue', ls='--', label='Design')
plt.legend()
plt.subplot(2,3,3)
plt.title("Az Max Jerk")
plt.hist(maxAzJerks, color='orange')
plt.xlabel("Jerk(deg/s^3)")
plt.ylim(0,120)
plt.plot([42.0, 42.0],[0,120],color='red', ls='--', label='Max')
plt.plot([28.0, 28.0],[0,120],color='blue', ls='--', label='Design')
plt.legend()
plt.subplot(2,3,4)
plt.title("El Max Velocity")
plt.hist(maxElVels, color='orange')
plt.xlabel("Velocity(deg/s)")
plt.ylim(0,120)
plt.plot([5.25, 5.25],[0,120],color='red', ls='--', label='Max')
plt.plot([3.5, 3.5],[0,120],color='blue', ls='--', label='Design')
plt.legend()
plt.subplot(2,3,5)
plt.title("El Max Accels")
plt.hist(maxElAccels, color='orange')
plt.xlabel("Accel(deg/s^2)")
plt.ylim(0,120)
plt.plot([5.25, 5.25],[0,120],color='red', ls='--', label='Max')
plt.plot([3.5, 3.5],[0,120],color='blue', ls='--', label='Design')
plt.legend()
plt.subplot(2,3,6)
plt.title("El Max Jerk")
plt.hist(maxElJerks, color='orange')
plt.xlabel("Jerk(deg/s^3)")
plt.ylim(0,120)
plt.plot([21, 21],[0,120],color='red', ls='--', label='Max')
plt.plot([14.0, 14.0],[0,120],color='blue', ls='--', label='Design')
plt.legend()

plt.savefig(f"/home/craiglagegit/DATA/MTMount_26jan23/Max_Accel_Jerks_26Jan23.pdf")

In [ ]:
Time(1679707649.0, format='unix_tai').isot

In [ ]:
time.unix_tai

In [ ]:
plotAz['timestamp'].values